### Load in packages and model

In [1]:
%cd ../../..

/home/nils/NILS/Master/DL2/DL2-ZeroVis


In [2]:
from src.fromage_inf.inf_utils import PromptParser
import pickle
import itertools
import torch
from PIL import Image
import nltk.translate.bleu_score as BLEU
from torchmetrics.multimodal.clip_score import CLIPScore


In [ ]:
parser = PromptParser("src/fromage_inf/fromage_model/")

# Load the relations dictionary to make the relations.
relations = pickle.load(open("src/code/relations_dict.pkl", "rb"))

# ZeroCap comparison

In [ ]:
# -Recall @ 5: indicates a word’s appearance within the first five words generated
# only use image name as reference : so US is wrong if USA is answer
# sklearn.metrics.recall_score :  y_truth = ['facebook'] y_pred = [5 first ouputs]

# Initialize the CLIP score metric with the same clip model as used in FROMAGe
C_metric = CLIPScore(model_name_or_path='openai/clip-vit-large-patch14')


total_scores = {}

for relation, values in relations.items():
    print(relation)
    print('=' * 120)

    relation_scores = {"CLIP-s": 0, "Recall@5": 0, "BLEU-1": 0}

    combinations = itertools.combinations(values, 2)

    amount_combi = len(combinations)

    for combo in combinations:
        tuple1, tuple2 = combo
         
        print('='*60)
        print("Arithmetic:")
        print("{} + ({} - {})".format(tuple1[0], tuple2[1], tuple2[0]))
        print("Expected result: {}".format(tuple1[1]))

        inp_image = parser.model.visual_embs[tuple1[0]] + (parser.model.visual_embs[tuple2[1]] - parser.model.visual_embs[tuple2[0]])

        # Add empty string to prevent error
        prompt = [inp_image]

        print('=' * 30)
        model_outputs = parser.model.generate_for_images_and_texts(prompt, ret_scale_factor=0.01, num_words=5)

        print('Model generated outputs:')
        parser.display(model_outputs)

        # SCORES
        ground_truth = tuple1[1].split("/")[1].replace("_"," ").split()

        # Recall @ 5
        recall_score = 
        relation_scores['Recall@5'] += recall_score/amount_combi

        # BLEU-1
        bleu_score = BLEU.sentence_bleu([ground_truth], model_outputs, (1./1.))
        relation_scores['BLEU-1'] += bleu_score/amount_combi

        # CLIP-s
        img = Image.open("src/benchmark/"+tuple1[1]+".jpg")
        img = img.resize((224, 224)).convert('RGB')
        clip_score = C_metric(img, "Image of a {}".format(model_outputs))
        relation_scores['CLIP-s'] += clip_score/amount_combi


        print('='*60)
        print("Arithmetic:")
        print("{} + ({} - {})".format(tuple2[0], tuple1[1], tuple1[0]))
        print("Expected result: {}".format(tuple2[1]))

        inp_image = parser.model.visual_embs[tuple2[0]] + (parser.model.visual_embs[tuple1[1]] - parser.model.visual_embs[tuple1[0]])

        # Add empty string to prevent error
        prompt = [inp_image]

        print('=' * 30)
        model_outputs = parser.model.generate_for_images_and_texts(prompt, ret_scale_factor=0.01, num_words=5)

        print('Model generated outputs:')
        parser.display(model_outputs)
    
    total_scores[relation] = relation_scores

print(total_scores)